In [ ]:
%load_ext lab_black

In [ ]:
import datetime as dt

import mplfinance as mpf
import pandas as pd
import numpy as np
import pandera as pa

import shared


logger = shared.get_logger()

In [ ]:
klines_btc_1m_df = shared.read_binance_klines()
klines_btc_1h_df = shared.split_df(df=klines_btc_1m_df, interval_str="1h")

In [ ]:
sundays_list = pd.date_range(start="2023-08-01", end="2024-03-30", freq="W").tolist()
weekeds_dt_ranges = [
    (sunday_dt - dt.timedelta(days=2), sunday_dt + dt.timedelta(days=1))
    for sunday_dt in sundays_list
]
for dt_start, dt_end in weekeds_dt_ranges:
    slice_df = klines_btc_1h_df[lambda x: x.index >= dt_start][
        lambda x: x.index <= dt_end
    ]
    fig, axlist = mpf.plot(
        slice_df,
        warn_too_much_data=len(slice_df),
        type="candle",
        figsize=(14, 3),
        style=shared.s,
        returnfig=True,
        vlines={
            "vlines": [(dt_start + dt.timedelta(days=1)).date()],
            "linewidths": 0.2,
        },
    )
    max_diff_abs = slice_df.high.max() - slice_df.low.min()
    max_diff_rel = (max_diff_abs / slice_df.low.min()) * 100
    x_label = f"[{dt_start.date()} - {dt_end.date()}] max_diff_rel={max_diff_rel:.2f}"
    axlist[-2].set_xlabel(x_label)
    axlist[-2].xaxis.set_ticklabels([])